<a href="https://colab.research.google.com/github/vamsikrishna00466/stock-market-anlalysis/blob/main/HEIKIN_ASHI_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [178]:
pip install mysql-connector-python-rf

In [179]:
#importing relevant libraries
import os
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import operator
import decimal
import time
import mysql.connector as mysql
from math import floor
import itertools
import warnings
warnings.filterwarnings('ignore')

In [181]:
df=df[['ins_date', 'open', 'high', 'low', 'close']]
df_original=df.copy()
#remove duplicates
df1 = df.drop_duplicates(keep='last')
df1=df1.reset_index()
df1=df1[['ins_date','open','high','low','close']]
#change the time zone
df1['ins_date'] = df1['ins_date'].dt.tz_localize('GMT').dt.tz_convert('Asia/Kolkata')


In [182]:
df1["date"] = list(map(lambda x: x[:19],df1["ins_date"].astype(str)))
df1["time"] = list(map(lambda x: x[11:],df1["date"].astype(str)))
df1["date1"] = list(map(lambda x: x[:10],df1["date"].astype(str)))


In [183]:
df2=df1.copy()
#df_drop=df1.loc[df1['date1'] == '2021-03-23']
df2 = df2.drop((df2.loc[df2['date1'] =='2021-03-23']).index)
df2 = df2.drop((df2.loc[df2['date1'] =='2021-03-22']).index)

In [184]:
df2=df2[['date','date1','open','high','low','close','time']]

In [185]:
df2['date'] = pd.to_datetime(df2.date)
df2 =df2.reset_index().set_index('date')

In [186]:
df2.tail()

,index,date1,open,high,low,close,time
date,,,,,,,
2021-03-19 15:25:00,572588,2021-03-19,34050.90,34070.15,34038.95,34069.80,15:25:00
2021-03-19 15:26:00,572589,2021-03-19,34069.10,34097.40,34067.20,34094.35,15:26:00
2021-03-19 15:27:00,572590,2021-03-19,34092.80,34098.50,34084.20,34089.15,15:27:00
2021-03-19 15:28:00,572591,2021-03-19,34085.75,34103.70,34083.80,34103.70,15:28:00
2021-03-19 15:29:00,572592,2021-03-19,34096.90,34124.30,34095.50,34115.10,15:29:00


In [187]:
#f3 = df2.between_time('09:15:00', '10:15:00')
#df4 =df3.reset_index(drop=True)
df3=df2.copy()
df3.reset_index(level=0, inplace=True)
df3 =df3[['date','date1','open','high','low','close','time']]
df3["Time"] = list(map(lambda x: x.time(),df3.date))

df3['open'] = df3['open'].astype('float')
df3['high'] = df3['high'].astype('float')
df3['low'] = df3['low'].astype('float')
df3['close'] = df3['close'].astype('float')

In [188]:
df3.tail()

,date,date1,open,high,low,close,time,Time
572588,2021-03-19 15:25:00,2021-03-19,34050.90,34070.15,34038.95,34069.80,15:25:00,15:25:00
572589,2021-03-19 15:26:00,2021-03-19,34069.10,34097.40,34067.20,34094.35,15:26:00,15:26:00
572590,2021-03-19 15:27:00,2021-03-19,34092.80,34098.50,34084.20,34089.15,15:27:00,15:27:00
572591,2021-03-19 15:28:00,2021-03-19,34085.75,34103.70,34083.80,34103.70,15:28:00,15:28:00
572592,2021-03-19 15:29:00,2021-03-19,34096.90,34124.30,34095.50,34115.10,15:29:00,15:29:00


In [189]:
df4 = df3.copy()

In [190]:
df4.tail()

,date,date1,open,high,low,close,time,Time
572588,2021-03-19 15:25:00,2021-03-19,34050.90,34070.15,34038.95,34069.80,15:25:00,15:25:00
572589,2021-03-19 15:26:00,2021-03-19,34069.10,34097.40,34067.20,34094.35,15:26:00,15:26:00
572590,2021-03-19 15:27:00,2021-03-19,34092.80,34098.50,34084.20,34089.15,15:27:00,15:27:00
572591,2021-03-19 15:28:00,2021-03-19,34085.75,34103.70,34083.80,34103.70,15:28:00,15:28:00
572592,2021-03-19 15:29:00,2021-03-19,34096.90,34124.30,34095.50,34115.10,15:29:00,15:29:00


In [191]:
ohlcv_dict = {
 'open': 'first',
 'high': 'max',
 'low': 'min',
 'close': 'last',
}

In [192]:
df4 =df4.reset_index().set_index('date')
df5 = df4.resample('60T', base=15).agg(ohlcv_dict)

In [193]:
df5.dropna(inplace=True)

In [194]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10695 entries, 2015-01-09 09:15:00 to 2021-03-19 15:15:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    10695 non-null  float64
 1   high    10695 non-null  float64
 2   low     10695 non-null  float64
 3   close   10695 non-null  float64
dtypes: float64(4)
memory usage: 417.8 KB


In [195]:
df5.tail()

,open,high,low,close
date,,,,
2021-03-19 11:15:00,34031.90,34062.75,33503.95,33544.65
2021-03-19 12:15:00,33545.70,33767.40,33479.30,33694.65
2021-03-19 13:15:00,33691.50,34030.30,33637.35,33920.70
2021-03-19 14:15:00,33925.50,34351.50,33767.55,34174.05
2021-03-19 15:15:00,34175.05,34185.85,33986.10,34115.10


In [196]:
#close_HEIKIN
n=0
df5['close_HEIKIN']=np.nan
for i in range(n,len(df5)):
  df5['close_HEIKIN'][i] =df5['close'][i]+df5['high'][i]+df5['low'][i]+df5['open'][i]
n=0
m=4
for i in range(n,len(df5)):
  df5['close_HEIKIN'][i] =df5['close_HEIKIN'][i]/m

#open_aHEIKIN
n=1
n=0
df5['open_HEIKIN']=np.nan
for i in range(n,len(df5)):
  df5['open_HEIKIN'][0] =df5['open'][0]
n=1
m=2
#for i in range(n,len(df5))
for i in range(n,len(df5)):
  df5['open_HEIKIN'][i] =df5['open_HEIKIN'][i-1]+df5['close_HEIKIN'][i-1] 
  df5['open_HEIKIN'][i] =df5['open_HEIKIN'][i]/m

#high_HEIKIN and low_HEIKIN
df5['high_HEIKIN']=np.nan
df5['high_HEIKIN'] = df5[['open_HEIKIN','close_HEIKIN','high']].max(axis=1)
df5['low_HEIKIN']=np.nan
df5['low_HEIKIN'] = df5[['open_HEIKIN','close_HEIKIN','low']].min(axis=1)

In [197]:
df5['candle_type']=np
n=0
for i in range(n, len(df5)):
  if df5['open_HEIKIN'][i]<df5['close_HEIKIN'][i]:
    df5['candle_type'][i]='Green'
  else:
    df5['candle_type'][i]='Red'



#df3["time"] = list(map(lambda x: x[11:],df3["date"].astype(str)))

In [198]:
df5.tail()

,open,high,low,close,close_HEIKIN,open_HEIKIN,high_HEIKIN,low_HEIKIN,candle_type
date,,,,,,,,,
2021-03-19 11:15:00,34031.90,34062.75,33503.95,33544.65,33785.8125,33848.134411,34062.750000,33503.950000,Red
2021-03-19 12:15:00,33545.70,33767.40,33479.30,33694.65,33621.7625,33816.973456,33816.973456,33479.300000,Red
2021-03-19 13:15:00,33691.50,34030.30,33637.35,33920.70,33819.9625,33719.367978,34030.300000,33637.350000,Green
2021-03-19 14:15:00,33925.50,34351.50,33767.55,34174.05,34054.6500,33769.665239,34351.500000,33767.550000,Green
2021-03-19 15:15:00,34175.05,34185.85,33986.10,34115.10,34115.5250,33912.157619,34185.850000,33912.157619,Green


In [199]:
df5.reset_index(level=0, inplace=True)


In [200]:
df5["time"] = list(map(lambda x: x[11:],df5["date"].astype(str)))
df5["date1"] = list(map(lambda x: x[:10],df5["date"].astype(str)))

In [201]:
df6 = df5.loc[df5['time']== '10:15:00']
df6=df6.reset_index()
df6=df6[['date','date1','time','open','high','low','close','open_HEIKIN','high_HEIKIN','low_HEIKIN','close_HEIKIN','candle_type']]

In [202]:
df6.tail()

,date,date1,time,open,high,low,close,open_HEIKIN,high_HEIKIN,low_HEIKIN,close_HEIKIN,candle_type
1521,2021-03-15 10:15:00,2021-03-15,10:15:00,34795.20,34899.90,34519.20,34559.75,35360.702289,35360.702289,34519.20,34693.5125,Red
1522,2021-03-16 10:15:00,2021-03-16,10:15:00,35110.15,35356.45,35096.65,35245.55,35096.862030,35356.450000,35096.65,35202.2000,Green
1523,2021-03-17 10:15:00,2021-03-17,10:15:00,34680.00,34838.55,34635.25,34756.25,34824.453610,34838.550000,34635.25,34727.5125,Red
1524,2021-03-18 10:15:00,2021-03-18,10:15:00,34599.80,34635.60,34408.80,34494.85,34475.796806,34635.600000,34408.80,34534.7625,Green
1525,2021-03-19 10:15:00,2021-03-19,10:15:00,33795.70,34122.00,33636.70,34036.65,33798.506323,34122.000000,33636.70,33897.7625,Green


In [203]:
#df6.to_excel("HEIKIN ASHI_final5.xlsx")